In [3]:
import pandas as pd
import yfinance as yf

IBOV

In [4]:
# Pegar dados do ibovespa
tickerData = yf.Ticker('^BVSP')
# Importar dados usando taxa de amostragem de 1 mês
tickerDf = tickerData.history(period='1mo', start='1900-12-31', end='2024-06-28')
# Deixar apenas data no indíce
tickerDf.index = tickerDf.index
ibov = tickerDf[['Close']]
ibov = ibov.rename_axis('DATE')
ibov.index = ibov.index.tz_localize(None)
ibov.to_csv('ibov.csv', sep=';', index=True, header=True)
print("Arquivo ibov.csv criado com sucesso!")
print(ibov.head())

Arquivo ibov.csv criado com sucesso!
                Close
DATE                 
1993-04-27  24.500000
1993-04-28  24.299999
1993-04-29  23.700001
1993-04-30  24.100000
1993-05-03  24.100000


IEE

In [ ]:
# Carregar o arquivo CSV usando o ponto e vírgula como separador
data = pd.read_csv("dados_iee_1997_2023.csv", encoding='ISO-8859-1', sep=';')
# Ajustar a criação da coluna 'Data'
data['Data'] = pd.to_datetime(data['Ano'].astype(str) + '-' + data['Mês'].astype(str) + '-01')
iee = data[['Data', 'Valor']]
iee = iee.set_index('Data')
iee.to_csv('iee.csv', sep=';',index=True, header=True)
print("Arquivo convertido com sucesso!")
print(iee.head())

INCC/IPCA/IGPM

In [ ]:
file_path = 'INCC_raw.csv'
data = pd.read_csv(file_path)
# Transformando as colunas de meses em linhas
data_melted = data.melt(id_vars=["Ano"], var_name="Mes", value_name="Valor")

# Convertendo a coluna 'Mes' para formato de data (considerando o primeiro dia de cada mês)
# e combinando com o ano
# Mapeamento dos nomes dos meses em português para inglês
meses_map = {
    'Jan': 'Jan', 'Fev': 'Feb', 'Mar': 'Mar', 'Abr': 'Apr', 'Mai': 'May', 'Jun': 'Jun',
    'Jul': 'Jul', 'Ago': 'Aug', 'Set': 'Sep', 'Out': 'Oct', 'Nov': 'Nov', 'Dez': 'Dec'
}

# Aplicando o mapeamento na coluna 'Mes'
data_melted['Mes'] = data_melted['Mes'].map(meses_map)

# Tentando novamente a conversão para data
data_melted['Data'] = pd.to_datetime(data_melted['Ano'].astype(str) + data_melted['Mes'], format='%Y%b')

# Organizando as colunas e removendo as colunas extras
data_final = data_melted[['Data', 'Valor']].sort_values(by='Data').reset_index(drop=True)
data_final = data_final.set_index(data_final.columns[0])

# Exibindo as primeiras linhas do dataframe final
data_final.head()
data_final.to_csv('INCC.csv')

SELIC

In [ ]:
import pandas as pd
import numpy as np

#importar .csv com os dados históricos da meta selic, taxa selic, TBC, TBAN e data das reuniões do COPOM
selic_raw = pd.read_csv('selic_raw.csv', header=0, sep=',', index_col=0) 

#criar dataframe a partir de três colunas (data e taxa a.a.) do dataframe df
only_selic = selic_raw.iloc[:,[0,6]]



#seperar dia, mes e ano da coluna data do dataframe only_selic
counter = 0
dia = list(range(0, len(only_selic['data'])))
mes_selic = list(range(0, len(only_selic['data'])))
ano_selic = list(range(0, len(only_selic['data'])))
for i in only_selic['data']:
    dia[counter], mes_selic[counter], ano_selic[counter] = str(i).split('/')
    counter += 1

#remover valores duplicados de mes e ano
mes_selic = list(dict.fromkeys(mes_selic))
ano_selic = list(dict.fromkeys(ano_selic))

#faz a transposta da lista ano
ano = np.transpose(ano_selic)
#utilizar metodo reverse na lista ano
ano = list(reversed(ano))


#criando lista de colunas
mes =['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto','Setembro', 'Outubro', 'Novembro', 'Dezembro']
selic = pd.DataFrame(index=mes, columns=ano)

#criar lista com as datas em ordem crescente
data_crescente = np.transpose(only_selic['data'])
data_crescente = list(reversed(data_crescente))

taxa_crescente = np.transpose(only_selic['taxa a.a.'])
taxa_crescente = list(reversed(taxa_crescente))

#criar dataframe padronizado para a taxa selic e suas datas
counter=0
last_value = None
last_month = None
for i in ano:
    for j in range(1,13):
        dia_comp, mes_comp, ano_comp = str(data_crescente[counter]).split('/')

        if(ano_comp == i and int(mes_comp) == j):
            last_value = taxa_crescente[counter]
            counter += 1

        #Encerrar o loop quando o contador for igual ao tamanho da lista data_crescente
        if(counter == len(data_crescente)):
            break
        #encontrar indice de i na lista ano
        index_ano = ano.index(i)
        selic.iloc[j-1, index_ano] = last_value

        #armazena os valores da próxima iteração casa haja mais de uma reunião no mesmo mês
        if(counter < 252):
            mes_comp_future   = str(data_crescente[counter]).split('/')[1]
            mes_comp_future_2 = str(data_crescente[counter+1]).split('/')[1]
            if(ano_comp == i and int(mes_comp_future) == j):
                counter += 1
                last_value = taxa_crescente[counter]
                
            if(ano_comp == i and int(mes_comp_future_2) == j):
                counter += 1
                last_value = taxa_crescente[counter]

print(selic)
selic.to_csv('selic.csv', sep=';', index=True, header=True)